# **Presentación de modelos predictivos**

## Requerimientos
1. Prediccion de variables climáticas con error cercano al 0%
2. En un lapso de tiempo de al menos 24 horas
3. Presentación de a lo menos cuatro modelos

**Opcionales:**

1. Generación de pronósticos a una semana
2. Determinación de condiciones como *nublado* o *soleado*

## Consideraciones

### R o Python
Si bien se posee mas experiencia de uso de R se decide ocupar Python puro para la creación de los modelos debido a la mayor disponibilidad de soporte y capacidad de paralelización de sus procesos (R sin paquetes externos no puede más de una núcleo por procesador)

### Sobre set de datos
De ahora en adelante se usara el término "datos locales" para hacer referencia a los datos creados mediante registros de los sensores UTA Mayor y datos DGAC a aquellos descargados desde la página de la Dirección de Aeronáutica Civil, del mismo modo, se ocupan las siglas **Ts, HR y QFE** para hacer referencia a las variables **Temperatura, Humedad relativa y Presión atmosférica** respectivamente, por **date** se hará referencia al **momento de captura** del registro

### *Multivariable* y *multipaso*
+ Un modelo multivariable es aquel que es dado por la fórmula: 
    > Var1 + Var2 = Var3 + Var4
+ Es decir, múltiples variables de entradas son usadas para definir varias variables de salida. Estos modelos son conocidos como **MIMO** (*multiple input, multiple output*)
+ Un modelo multipaso es aquel dado por la fórmula: 
    > Var1 = Var1(n-1) + Var1(n-2) + ... + Var1(0)
+ Es decir, múltiples estados pasados de una variable son usados para definir el estado actual o futuro de la misma variable. Estos model son denominados **Multistep** 

## Contexto
La confección de pronósticos climáticos es una disciplina antigua, pero que no se ejercía de forma metódica y precisa hasta la segunda mitad del siglo pasado, producto del empleo de herramientas como globos climáticos y comunicación por radio lo cual facilitaba el registro veraz y coordinado de ciertas variables atmosféricas en distintas zonas geográficas.
Construída esta base es que Edward Lorentz definió tres posibles paradigmas para la confección de pronósticos climáticos:

+ **Acercamiento dinámico**: El cual comprende la atmósfera como un gas el cual obedece las leyes de la física, y que mediante la definición de las ecuaciones que le gobiernan, se podrían determinar sus estados futuros y sus cambios conforme a alteraciones en sus variables **de entrada**. La desventaja de este paradigma es su lentitud y alto costo, pues requiere la acción coordinada de varias estaciones, en tiempo real, y poder computacional
+ **Acercamiento empírico**: Asume que las condiciones futuras del clima futuro serán similares a las presentadas en el pasado
    - Por ejemplo: Que el promedio de temperaturas de febrero del próximo año, será igual al febrero de este año
    Mediante este paradigma se ampara el uso de ML para la creación de pronósticos, pues en base a eventos pasados, se puede generalizar con precisión el comportamiento de variables climáticas en el futuro. La desventaja de este enfoque es su incapacidad de predecir fenómenos no registrados o de rara ocurrencia
+ **Acercamiento dinámico-empírico**: Fusión de los paradigmas anteriores, es el acercamiento *de facto* para la creación de pronósticos, hasta hace unas décadas era representado por el algoritmo MS5, ahora por el WFR

### Importado de librerías y cargando funciones de utilidad

In [307]:
# Uso universal
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas
import sqlalchemy
import pymysql
import joblib
from math import sqrt
from sklearn.metrics import mean_squared_error

# 1er modelo
from statsmodels.tsa.arima.model import ARIMA
# 2do y cuarto modelo
import tensorflow as tf
# 3er modelo
from sklearn.ensemble import RandomForestRegressor
# import joblib
def createTimeFeatures (dfToExpand):
    dfToExpand['minute'] = pandas.to_datetime(dfToExpand['utc']).dt.minute
    dfToExpand['hour'] = pandas.to_datetime(dfToExpand['utc']).dt.hour
    dfToExpand['day'] = pandas.to_datetime(dfToExpand['utc']).dt.day
    dfToExpand['month'] = pandas.to_datetime(dfToExpand['utc']).dt.month
    dfToExpand['year'] = pandas.to_datetime(dfToExpand['utc']).dt.year
    return dfToExpand

In [308]:
try:
    credentials = np.genfromtxt("pass",dtype='str')
    engine = sqlalchemy.create_engine("mysql+pymysql://"+credentials[0]+":"+credentials[1]+"@"+credentials[2]+"/"+credentials[3] )
    mydb = engine.connect()
    # query = "SELECT * FROM WEATHER_MEASUREMENT WHERE serverDate >= '2021-06-26 00:00:00' AND serverDate <= '2021-08-05';"
    query = "SELECT * FROM WEATHER_MEASUREMENT ORDER BY ID DESC LIMIT 34560;"
    arimadf = pandas.read_sql(query,mydb)
except:
    mydb.close() 
    print("error conexion a db")

# **Iteración uno: Modelo RF preliminar**
Con el fin de obtener de forma inmediata una línea base de efectividad a mejorar mediante la creación de nuevos modelos a la vez que un acercamiento al uso de Python para el ML, es que se inicia este proceso construyendo un bosque aleatorio **(RF)** simple que consuma los datos locales de forma cruda, este algoritmo es de especial utilidad para esta tarea puesto a que no requiere la afinación de hiperparámetros (como si los requeriría una red neuronal **-DNN-**)

## Preparación de datos
Esta iteración se destaca por el escaso preprocesamiento de datos al alimentar el modelo

**Selección de datos**: Datos completamente locales (~100.000 aprox.)

**Limpieza de datos**: No aplica, por el contrario no existen valores nulos en los datos locales pues son generados mediante un proceso metótico y automatizado (sensores) por lo tanto de esta iteración en adelante se omite este paso

## Modelado
+ Fórmula: Ts ~ HR + QFE + date
+ Algoritmo seleccionado: RF

## Evaluación
El modelo no se pudo concretar
+ El consumo de recursos computacionales es excesivo
+ La predicción de una variable climática cada 5 segundos puede resultar innecesaria considerando que se ha de generar pronósticos a una semana en el futuro
    - Esto resultaría en 168\*60\*12 predicciones, es decir ~121.000 registros por modelo
+ Los *predictores* (HR + QFE + date) están mal definidos, pues para determinar el valor de una variable climática en el futuro solo se cuenta con un *momento* (e.g: '2021-08-08 12:00:00) y no con valores de HR o QFE

Este problema se puede solucionar eliminando del set de datos de entrenamiento las variables HR y QFE, pero genera otros inconvenientes.
Solo se cuenta con datos tipo fecha, los cuales en este formato se comportan como un escalar (e.g: valores de 0 a 1000 con pasos de uno en uno) lo cual puede no ser útil para la detección de patrones por parte de los modelos. Ante esta situación se plantean tres opciones:
    
1. Si se asume que para alimentar al modelo solo se cuenta con **momentos (o fechas)** pasadas para pronósticos futuros, se podría aplicar análisis de **series de tiempo** en los datos, y por tanto todas las herramientas ya desarrolladas para este tipo de fenómenos
    > Fórmula: Ts(n) ~ Ts(n-1) + Ts(n-2) + ...
2. Por otra parte, la descomposición de fechas en **características de tiempo**, es decir, minutos, horas, días y meses, es intuitivamente útil, pues si el modelo logra relacionar por ejemplo el promedio de temperaturas de julio con un valor más bajo que el promedio de junio, se obtendría un pronóstico básico, pero funcional
    > Fórmula: Ts(n) ~ H + D + M 
3. Por último, apegándose mas a la definición de *acercamiento empírico*, se asume que las condiciones climáticas futuras son definidas de forma importante por las condiciones pasadas, por lo tanto se puede construir un **mapeo** de una serie de condiciones pasadas (incluyendo HR y QFE) a un momento futuro de éstas mismas variables
    > Fórmula: Ts(n) ~ Ts(n-1) + HR(n-1) + QFE(n-1)

> *Nota*: Todas estas fórmulas pueden aplicarse en su forma multivariable (**MIMO**) en cuanto el algoritmo a ocupar lo permita, por lo tanto una fórmula del tipo (Ts+HR+QFE)(n) ~ H + D + M es perfectamente válida

# **Iteración dos: Modelo DNN**
Tomando en cuenta la evaluación del modelo anterior es que en esta iteración se comienza a hacer preprocesamiento de datos, proceso el cual también conlleva a tomar decisiones en un contexto de **costo-beneficio**, pues la construcción y formateo de datos estará alterando la ***realidad*** registrada por los sensores en pos de alimentar a los modelos ML con información la cual sea consistente y eficiente de procesar en vez de *real*. La estrategia de preprocesamiento en esta iteración corresponde a una mezcla entre las opciones **2 y 3** de la sección anterior

## Preparación de datos
**Selección de datos**: Para disminuir el uso de recursos computacionales se decide hacer **agrupamiento de los registros** por minuto, disminuyendo 12 veces el tamaño del dataset original. 

**Construcción**: El agrupamiento de datos por medio de minutos reveló **gaps** entre los registros, los cuales deben ser rellenados si se desea aplicar el método 3 pues, por ejemplo, una temperatura de las 8am podría ser definida por la Ts de las 1am, último Ts registrado antes de un periodo de baja del servicio. En python esta operación se denomina **interpolado**

**Formateo**: Para las redes neuronales, este es un paso obligatorio, las transformaciones que han de aplicarse para alimentar una DNN de *tensorflow* incluye:

+ Descomposición de tiempo: La DNN es incapaz de procesar datos tipo *date*    
+ Interpolado: Es incapaz de procesar datos nulos
+ Normalización: Las DNN son sensibles a los valores fuera de un rango (-1,1) pues valores muy altos pueden falsamente anunciarse como cambios de alta importancia y propagarse entre las neuronas a pesar de ser simplemente el promedio de una variable (e.g: Ts vs QFE)

## Modelado
+ Características del modelo: Mutlivariable
+ Fórmula: Ts(n) ~ Ts(n-1) + min + H + D + M
+ Algoritmo seleccionado: DNN, 4 capas densas, Entradas \[7,] -> Salida\[3]

In [309]:
# Preparación de datos
## Seleccion
df = pandas.read_csv("./WEATHER_MEASUREMENT.csv")
df["utc"] = pandas.to_datetime(df["dateUTC"],format='%Y-%m-%d %H:%M:%S')
df = df.groupby(pandas.Grouper(key="utc",freq='min')).mean()    
toKeep = ['AMBIENT_TEMPERATURE','AIR_PRESSURE','HUMIDITY']
df = df[toKeep]
df = df.reset_index()
df.columns = ['utc','Ts_Valor','QFE_Valor','HR_Valor']
df = df[['Ts_Valor','HR_Valor','QFE_Valor','utc']]

In [310]:
## Construcción
df = df.interpolate(method = 'linear')
## Formateo
df = createTimeFeatures(df)
dates_df = df.pop('utc')
df.pop('year')
## Normalizado
train_mean = df.mean()
train_std = df.std()
df = (df - train_mean) / train_std

In [311]:
# Modelado
## Creación de *labels* para utilizar como resultados de fórmula
topred = df[['Ts_Valor','HR_Valor','QFE_Valor']]
topred = topred[1:]
topred = topred.reset_index(drop=True)
# topred = (topred - train_mean[['Ts_Valor','HR_Valor','QFE_Valor']]) / train_std[['Ts_Valor','HR_Valor','QFE_Valor']]
df = df[:-1]

In [312]:
## Creación datasets de entrenamiento y testeo
lendf = len(df)
lendf = round(lendf*0.9)
train_df = df[0:lendf]
train_labels = topred[0:lendf]
test_df = df[lendf:len(df)]
test_labels = topred[lendf:len(df)]

In [313]:
topred.tail()
topred.shape

(49012, 3)

In [314]:
df.tail()
df.shape

(49012, 7)

In [315]:
# Modelado
features = test_df.shape[1]
train_df = np.array(train_df)
test_df = np.array(test_df)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

nnHM = tf.keras.Sequential()
nnHM.add(tf.keras.layers.Dense(168,input_shape=(features,)))
nnHM.add(tf.keras.layers.Dense(72,input_shape=(features,)))
nnHM.add(tf.keras.layers.Dense(24,activation='relu'))
nnHM.add(tf.keras.layers.Dense(3))
# nnHM.predict(test_df)

In [316]:
## Compilado de modelo
nnHM.compile(
    optimizer=tf.optimizers.Adam(),
    #learning_rate = 0.001
    loss='mae', # 'mean_absolute_error',
    metrics=[tf.keras.metrics.MeanSquaredError()] 
)
history = nnHM.fit(
    train_df, train_labels,
    epochs=3,
    verbose=1,
    shuffle=False,
    validation_split=0.15
)

Epoch 1/3
1172/1172 [==============================] - 14s 11ms/step - loss: 0.1573 - mean_squared_error: 0.0894 - val_loss: 0.4396 - val_mean_squared_error: 0.5600
Epoch 2/3
1172/1172 [==============================] - 13s 12ms/step - loss: 0.1031 - mean_squared_error: 0.0307 - val_loss: 0.2169 - val_mean_squared_error: 0.0921
Epoch 3/3
1172/1172 [==============================] - 13s 11ms/step - loss: 0.0806 - mean_squared_error: 0.0235 - val_loss: 0.2779 - val_mean_squared_error: 0.1230


In [317]:
x = nnHM.predict(test_df)
stackPreds = pandas.DataFrame(x)
stackPreds.columns = ['Ts_Valor','HR_Valor','QFE_Valor']
test_labels = pandas.DataFrame(test_labels)
test_labels.columns = ['Ts_Valor','HR_Valor','QFE_Valor']
test_labels = test_labels.reset_index(drop=True)

In [318]:
#### NO CORRER
# De normalizacion
stackPreds = stackPreds * train_std + train_mean
test_labels = test_labels * train_std + train_mean
#stackPreds.plot(subplots=True)
#test_labels.plot(subplots=True)
#plt.show()

## Evaluación
A primera vista pareciera que el modelo tiene una alta capacidad predictiva debido a los errores promedio cercanos a cero considerando la varianza original de los datos. Lo cual se confirma al apreciar la cercanía entre las líneas dibujadas por las predicciones y el dataset de testeo

In [319]:
%matplotlib qt
plt.plot(stackPreds['Ts_Valor'],label="Predicciones DNN")
plt.plot(test_labels['Ts_Valor'],label="Test Labels")
plt.title("Predicciones de temperatura - Modelo DNN - Fórmula: (Ts+HR+QFE)(n) ~ (Ts+HR+QFE)(n-1) + TimeFeatures")
plt.ylabel("Temperatura (°C)")
plt.xlabel("Cantidad de minutos en el futuro")
plt.legend()
plt.grid()
plt.show()

In [320]:
a = stackPreds - test_labels
a.mean()

HR_Valor     0.034780
QFE_Valor    2.160599
Ts_Valor     0.074774
day               NaN
hour              NaN
minute            NaN
month             NaN
dtype: float64

## Implementación
Obedeciendo la fórmula de este modelo: **Ts(n) ~ Ts(n-1) + min + H + D + M**

Es que las predicciones que este genera deben ser retroalimentadas como *input* para la generación de una siguiente predicción o **paso**, en este momento es donde el modelo falla de forma evidente como se puede ver en la siguiente gráfica

In [321]:
r = [18.955,55.959,1012.699,0,0,1,8]
r = pandas.DataFrame(r).transpose()
r.columns = ['Ts_Valor','HR_Valor','QFE_Valor','minute','hour','day','month']
r = (r-train_mean)/train_std
r = r[['Ts_Valor','HR_Valor','QFE_Valor','minute','hour','day','month']]
r

,Ts_Valor,HR_Valor,QFE_Valor,minute,hour,day,month
0,-0.564649,1.005445,1.379337,-1.703241,-1.662508,-1.770372,3.187065


In [322]:
now = pandas.to_datetime('2021-07-29 18:00:00')
features = test_df.shape[1]
x = nnHM.predict(r)
stackPreds = pandas.DataFrame()
for i in range(20):
    delta = now + datetime.timedelta(0,i*60)
    #temp = np.array([x,y,z,delta.hour,delta.day,delta.month],dtype="float32")
    temp = np.array([x[0,0],x[0,1],x[0,2],delta.minute,delta.hour,delta.day,delta.month],dtype="float32")
    stackPreds = stackPreds.append(pandas.DataFrame(temp).transpose())
    # Formateando registro para alimentar el modelo
    temp_RE = np.reshape(temp,(-1,features))
    x = nnHM.predict(temp_RE)

stackPreds.columns = ['Ts_Valor','HR_Valor','QFE_Valor','minute','hour','day','month']
stackPreds = stackPreds * train_std + train_mean
stackPreds = stackPreds.reset_index(drop=True)

In [323]:
test_labels = test_labels.reset_index(drop=True)
plt.plot(stackPreds['Ts_Valor'],label="Predicciones DNN")
plt.plot(test_labels[['Ts_Valor']],label="Test Labels")
plt.title("Predicciones de temperatura - Modelo DNN - Fórmula: (Ts+HR+QFE)(n) ~ (Ts+HR+QFE)(n-1) + TimeFeatures")
plt.ylabel("Temperatura (°C)")
plt.xlabel("Cantidad de minutos en el futuro")
plt.legend()
plt.grid()
plt.show()

La falla de este modelo recae en su fórmula **Ts(n) ~ Ts(n-1) + min + H + D + M** , la cual indica que todas las predicciones son mayormente dependientes de sus valores inmediatamente anteriores (Ts(n) ~ Ts(n-1)) y además al hecho de haber aprendido en mayor parte el **comportamiento lineal** de los datos, el cual fue inducido al momento de hacer interpolado con tal de obtener un dataset consistente.

Ante la incapacidad de los modelos de retroalimentarse y generar predicciones fiables no tan dependientes de los registros inmediatamente anteriores captados se idea una serie de formas posibles de apalear los efectos de este fenomeno y se comentan sus posible desventajas:
+ **No proveer de la varible minuto a los registros**: Considerando que las predicciones:
    1. O son incapaces de entender los minutos como un simple componente de hora, es decir que los minutos de la hora 16 tienen distinto comportamiento que los minutos de la hora 3 am
    2. O la variable minuto simplemente provee rigidez al modelo, debido a su comportamiento lineal, lo cual lo hace incapaz de reconocer la mayor importancia de mapear en 1er lugar e.g: hora -> temperatura
+ Agrandar los gaps entre los registros: A hora o días, pues los comportamientos lineales de los tiempos de downtime son *extensos* considerando la granularidad de los registros, lo cual afecta negativamente la deteccion de periodicidad en los datos
    > Una iteración 2.5 demostró que esta solución no es viable
+ **Mejorar interpolacion**: Preferentemente mediante una funcion sinusoidal que imite los ciclos naturales diarios de las variables climáticas
    1. Imposible aplicar para humedad relativa con datos locales, su comportaminento es muy errático
    2. Disponibilidad: se requiere al menos un par de dias de registros sin downtime o integración de datos externos
    3. Alta complejidad: pues requeriria ya un modelo de prediccion funcional tan solo para hacer preprocesamiento
+ **Integración de datos**: Dgac y locales, sobreescribiendo los datos locales en los datos Dgac, de esta forma los modelos tienen mayor espacio para aprender los ciclos de las variables
    1. Se perderia una enorme cantidad de datos (720 veces menos datos)
+ Mapeos mas extensos: Conocido también como **sistema de ventanas**, consiste en relacionar una mayor cantidad de datos por cada *label*
    1. e.g: matriz6horasDeDatos[][] -> prediccion1hora[]

Tomando en cuenta los aprendizajes obtenidos mendiante la visualización de datos es que se opta por aplicar soluciones incluyendo los puntos 1, 4 y 5, y retomando las opciones 1 y 2 mencionadas en la iteración uno, las cuales no empleaban variables que requieren retroalimentación, pues dependen únicamente de la fecha de los registros y comportamientos cíclicos *univariables*

# **Iteración tres: Modelo ARIMA**
Un modelo autorregresivo de promedio móvil integrado (ARIMA) es un modelo estadístico que utiliza variaciones y regresiones de datos estadísticos con el fin de encontrar patrones para una predicción hacia el futuro [\[1\]](https://es.wikipedia.org/wiki/Modelo_autorregresivo_integrado_de_media_m%C3%B3vil) .

Este tipo de modelos son *univariables* pero requieren que el usuario indique, como mínimo, la periodicidad percibida de la serie de tiempo a estudiar, como se conoce que la mayoría de variables climáticas dibujan ciclos de 24 horas de acuerdo a las horas del día, se puede construir un modelo ARIMA por cada variable [\[2\]](https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/).

## Preparación de datos
**Selección de datos**: Últimos dos días de registros locales (~35.000 aprox.) agrupados por hora.
> Una iteración 3.5 demostró que el uso de dataset de dataset agrupados por minuto genera una excesiva carga computacional y tamaño de modelo (~ 2 gb) 

**Construcción**: Interpolado lineal 

**Formateo**: Tarea extensa debido al tipo de index el cual consume el modelo, que debe ser de tipo *date* con frecuencia horaria explicitada, el cual además para subir a DB, debe pasar por un proceso inverso.

## Modelado
+ Fórmula: Ts(n) = Ts(n-1) + Ts(n-2) + ... + Ts(0)
+ Algoritmo: ARIMA del paquete 'statsmodels' (en rigor es un SARIMAX), periodicidad cada 24 registros y grado de integración 1.

In [327]:
df = df.iloc[::-1] # dando vuelta el dataframe
df = arimadf.iloc[::-1] # dando vuelta el dataframe
df["utc"] = pandas.to_datetime(df["serverDate"],format='%Y-%m-%d %H:%M:%S')
df = df[['AMBIENT_TEMPERATURE','AIR_PRESSURE','HUMIDITY','utc']]
df = df.reset_index(drop=True)
df2 = df.groupby(pandas.Grouper(key="utc",freq='H')).mean()
df2 = df2.interpolate(method='linear')
series = df2
series.index = series.index.to_period('H')

<ipython-input-327-44c49a53ab8d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["utc"] = pandas.to_datetime(df["serverDate"],format='%Y-%m-%d %H:%M:%S')


Una de las mejores caracteristicas de los modelos ARIMA (o al menos su implementacion en pyhon) es que puede hacer bulk de predicciones con tan solo un valor futuro fijo de la serie de tiempo (o sea una fecha futura arbitraria), sin embargo al mismo tiempo dependera complemente de los ultimos steps que el mismo modelo va creando, los cuales van disminuyendo a través del tiempo, convergiendo las prediccioes en el promedio de las variables.


In [325]:
future = 72

stackPreds = pandas.DataFrame()
model = ARIMA(series['AMBIENT_TEMPERATURE'], order=(24,1,0))
model_fit = model.fit()
stackPreds['AMBIENT_TEMPERATURE'] = model_fit.forecast(future)
model = ARIMA(series['HUMIDITY'], order=(24,1,0))
model_fit = model.fit()
stackPreds['HUMIDITY'] = model_fit.forecast(future)
model = ARIMA(series['AIR_PRESSURE'], order=(24,1,0))
model_fit = model.fit()
stackPreds['AIR_PRESSURE'] = model_fit.forecast(future)

/usr/lib/python3.9/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


## Evaluación

In [326]:
series.plot(subplots=True)
plt.show()

In [329]:
stackPreds.plot(subplots=True)
plt.show()

Como se puede observar, el modelo genera gráficas que en mayor parte se asemejan a los comportamientos de las variables que le fueron proveídas, el principal objetivo de esta iteración fue alcanzado pues se construyó un modelo el cual puede consumir sus propias predicciones como input y dar *pasos en el futuro* de forma estable

# **Iteración cuatro: Modelo DNN - solo tiempo**
Continuando con la premisa del modelo ARIMA (es decir, el valor actual de las variables dependen únicamente de sus momentos pasados) es que mediante un modelo DNN, adaptamos esta premisa a los algoritmos de aprendizaje automáticos modernos. De esta forma, de forma computacional, la variable a predecir ya no consume un bulk de sus estados pasados, sino que se construye un mapeo 
> Momento (pasado o futuro) -> Variable

El cual generalize mejor los mapeos "momento -> variable" pasados. De esta forma los nuevos *inputs* del modelo se convierten en **características de tiempo** como hora, día y mes, los cuales siempre existirán y no requieren que el modelo consuma sus propias predicciones
A diferencia de ARIMA, un DNN construído de forma manual en *tensorflow* puede ser de *output* multivariable, y además se puede controlar con precisión los comportamientos internos de la red neuronal como la cantidad de capas y forma de activación.

## Preparación de datos
**Selección de datos**: Proveer de datos consistentes y extensos es de vital importancia en el aprendizaje automático por lo que se emplean todos los registros locales disponibles agrupados por hora y además se hace selección de datos externos para complementar

**Construcción**: Sabiendo que los gaps de nuestros datos locales van a ser suplidos por externos, este paso no aplica de aquí en adelante

**Integración**: Se integran datos externos proveídos por DGAC de 2019 y 2020 para rellenar nuestros registros faltantes, este paso se aplica para todas las iteraciones de aquí en adelante

**Formateo**: Incluye descomposición de tiempo y normalización 

## Modelado
+ Fórmula: Ts(n) ~ H + D + M , en su versión multivariable (Ts+HR+QFE)(n) ~ H + D + M
+ Algoritmo: Stack de 5 capas de redes neuronales densas (*fully connected neurons* sin métodos internos)

In [330]:
df = pandas.read_csv("./dataPreprocessed.csv")
df = createTimeFeatures(df)
df = df[['Ts_Valor','HR_Valor','QFE_Valor','hour','day','month']]

lendf = len(df)
lendf = round(lendf*0.998)
train_df = df[0:lendf]
test_df = df[lendf:len(df)]

varToPred = 'Ts_Valor'
varToPred = ['Ts_Valor','HR_Valor','QFE_Valor']
train_labels = df[varToPred][0:len(train_df)]
test_labels = df[varToPred][lendf:]

# salvando valores de test para autoregresiones
rtTest = test_df.copy()
rtTest_labels = test_labels.copy()

train_mean = train_df.mean()
train_std = train_df.std()
train_df = (train_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std
train_labels = (train_labels - train_mean[varToPred]) / train_std[varToPred]
test_labels = (test_labels - train_mean[varToPred]) / train_std[varToPred]

train_df = train_df[['hour','day','month']]
test_df = test_df[['hour','day','month']]
features = train_df.ndim + 1

In [331]:
train_df = np.array(train_df)
#test_df = np.array(test_df)
train_labels = np.array(train_labels)
#test_labels = np.array(test_labels)

nnHM = tf.keras.Sequential()
nnHM.add(tf.keras.layers.Dense(672,input_shape=(3,)))
nnHM.add(tf.keras.layers.Dense(168,activation='relu'))
nnHM.add(tf.keras.layers.Dense(72,activation='relu'))
nnHM.add(tf.keras.layers.Dense(24,activation='relu'))
nnHM.add(tf.keras.layers.Dense(3))

In [333]:
nnHM.compile(
    optimizer=tf.optimizers.Adam(),
    #learning_rate = 0.001
    loss='mae', # 'mean_absolute_error',
    metrics=[tf.keras.metrics.MeanSquaredError()] # you were running metrics with 'accuracy' lol
)
history = nnHM.fit(
    train_df, train_labels,
    epochs=2,
    verbose=1,
    shuffle=False,
    validation_split=0.15
)

Epoch 1/2
832/832 [==============================] - 14s 16ms/step - loss: 0.5516 - mean_squared_error: 0.5189 - val_loss: 0.8333 - val_mean_squared_error: 1.0421
Epoch 2/2
832/832 [==============================] - 14s 17ms/step - loss: 0.5443 - mean_squared_error: 0.5137 - val_loss: 0.8117 - val_mean_squared_error: 1.0124


In [334]:
stackPreds.plot(subplots=True)
test_labels.plot(subplots=True)
plt.show()

In [335]:
#x = nnHM.predict(test_df)
nnHM = tf.keras.models.load_model('../deepNN-multiV2')
x = nnHM.predict(test_df)

stackPreds = pandas.DataFrame(x)
stackPreds.columns = ['Ts_Valor','HR_Valor','QFE_Valor']
test_labels = pandas.DataFrame(test_labels)
test_labels.columns = ['Ts_Valor','HR_Valor','QFE_Valor']

In [336]:
stackPreds = stackPreds * train_std + train_mean
test_labels = test_labels * train_std + train_mean
stackPreds = stackPreds.reset_index(drop=True)
test_labels = test_labels.reset_index(drop=True)

## Evaluación

In [337]:
plt.plot(stackPreds['Ts_Valor'],label="Predicciones")
plt.plot(test_labels['Ts_Valor'],label="Test Labels")
plt.title("Predicciones de temperatura - Modelo DNN - Fórmula: Ts ~ h + d + m")
plt.ylabel("Temperatura (°C)")
plt.xlabel("Horas en el futuro desde el 31-07 03:00")
plt.legend()
plt.grid()
plt.show()

In [338]:
plt.plot(stackPreds['HR_Valor'],label="Predicciones")
plt.plot(test_labels['HR_Valor'],label="Predicciones")
plt.legend()
plt.grid()
plt.title("Predicciones de humedad relativa - Modelo DNN - Fórmula: HR ~ h + d + m")
plt.ylabel("Humedad relativa (%)")
plt.xlabel("Horas en el futuro desde el 31-07 03:00")
plt.show()

In [339]:
a = ((stackPreds - test_labels) * (stackPreds - test_labels)).mean()
a

HR_Valor     42.595942
QFE_Valor     3.570640
Ts_Valor      0.897967
day                NaN
hour               NaN
month              NaN
dtype: float64

Como se puede observar, el modelo genera gráficas que en mayor parte se asemejan a los comportamientos de las variables en el set de datos de testeo, sin embargo cabe mencionar ciertas observaciones a mejorar en futuras iteraciones:
+ Las predicciones tienen un comportamiento de **señal** más que alguno de fenómeno natural. Esto se debe a que:
    - El modelo consume variables estáticas, es decir a través de los años los componentes **H, D y M** siempre tienen el mismo comportamiento (ir de 1 a 24, 1 a 30 y 1 a 12 respectivamente), por lo que en resumen, el modelo no hace mas que **mapear los mejores promedios a distintos momentos del año**
    - Este fenómeno era esperable dada la fórmula que define el modelo, el cual consume valores constantes
+ Los valores de error promedio calculado van en conjunto con lo expuesto, pues en visualizaciones de datos preliminares ya se observaba que las variables Ts y QFE tenían un comportamiento cíclico rígido y, por el contrario, HR era la variable con más ruido

De esta forma, esta iteración confirma la **importancia** que tiene el alimentar al modelo con registros de variables pasadas, esta vez, abasteciendo con suficientes datos históricos con diferencias significantes para privilegiar la detección de **ciclos (o estacionalidad)** y en segundo lugar el efecto de registros inmediatemente anteriores sobre la predicción final.

> Una solución alternativa, pero que conceptualmente no es diferente a lo desarrollado en esta iteración, es el forzar el comportamiento de señal en las variables, mediante la aplicación de transformaciones tipo **H = sin(H)** y **D + M = sin(D + M)** con tal de explicitar los ciclos diarios y anuales de cada variable

# **Iteración cinco: Modelo RF**
Tomando en cuenta la evaluación de la iteración pasada, es que esta vez el modelo a generar debe captar ciclos de variables a la vez que ser 
sensible a los valores inmediatamente anteriores de otras condiciones climáticas.
Por lo que para esta iteración se retoma la fórmula Ts(n) ~ Ts(n-1) + min + H + D + M pero eliminando el componente minutos e incrementando la cantidad de información de entrenamiento mediante la integración de datos de DGAC

La afinación de una regresión y el modelo detrás de este es un proceso costoso computacionalmente a la vez que inestable, problemas propios de las redes neuronales como *exploding/vanishing gradient* fueron presensiados en iteraciones pasadas pero no mencionadas para mantener el *notebook* fluido, al mismo tiempo, las diferentes formas de normalizar e interpolar datos añaden mayor complejidad y variables que deben ser meticulosamente traceadas y guardadas al momento de creación de un modelo NN
Ante estas dificultades es que se vuelve a aplicar un modelo RF, el cual no presenta los problemas descritos pues simplemente divide y promedia la serie de árboles de decisión que construye para generar una regresión

## Preparación de datos
**Selección de datos**: Datos locales y DGAC (2 años) agrupados por hora

**Integración**: Se escribe nuestros datos locales sobre los registros DGAC mediante una operación *join&replace*

**Formateo**: Descomposición de tiempo

## Modelado
+ Fórmula: Ts(n) ~ Ts(n-1) + HR(n-1) + QFE(n-1) + H + D + M + Y
+ Algoritmo seleccionado: RF, su implementación en python permite *output* multivariable (Entradas \[7,] -> Salida\[3]) a la vez que mapeo por ventanas

In [340]:
# Seleccion e integración
df = pandas.read_csv("./dataPreprocessed.csv")
df = createTimeFeatures(df)
df.pop('minute')
dates_df = df.pop('utc')

# Preparación de datos y labels para cumplir con fórmula
topred = df[['Ts_Valor','HR_Valor','QFE_Valor']]
topred = topred[1:]
df = df[:-1]

# Dividiendo dataset
lendf = len(df)
lendf = round(lendf*0.9)
train_df = df[0:lendf]
train_labels = topred[0:lendf]
test_df = df[lendf:len(df)]
test_labels = topred[lendf:len(df)]

In [341]:
rfabc = RandomForestRegressor()
# rfabc.fit(train_df,train_labels)
rfMultivar=joblib.load('../../pyRF/rf-multivarOffset.joblib')

predRF = rfMultivar.predict(test_df)
test_df = test_df.reset_index(drop=True)

In [342]:
#pandas.DataFrame(predRF).plot(subplots=True)
#test_labels.plot(subplots=True)
#plt.show()

## Evaluación

In [343]:
a = ((predRF - test_df[['Ts_Valor','HR_Valor','QFE_Valor']]))
a.mean()

Ts_Valor     0.095336
HR_Valor     0.051294
QFE_Valor   -0.002888
dtype: float64

Como se puede observar, el modelo genera gráficas que se asemejan mucho a los comportamientos de las variables en el set de datos de testeo, sin embargo, esto es solo una línea base, la principal misión de este modelo es demostrar la capacidad de consumir sus propias predicciones conservando la estacionalidad de las variables climáticas, de esta forma se genera un nuevo plan de evaluación el cual desarrolla esta procedimiento, resultando en las siguientes predicciones:

In [344]:
now = datetime.datetime.now()
now = pandas.to_datetime('2021-07-29 09:00:00')
stackPreds = pandas.DataFrame()
x = rfMultivar.predict(train_df[len(train_df)-1:])

In [345]:
for i in range(0,72):
    delta = now + datetime.timedelta(0,i*3600)
    #temp = np.array([x,y,z,delta.hour,delta.day,delta.month],dtype="float32")
    temp = np.array([x[0,0],x[0,1],x[0,2],delta.hour,delta.day,delta.month,delta.year],dtype="float32")
    stackPreds = stackPreds.append(pandas.DataFrame(temp).transpose())
    # Formateando registro para alimentar el modelo
    temp_RE = np.reshape(temp,(-1,7))
    x = rfMultivar.predict(temp_RE)

In [346]:
stackPreds = stackPreds.reset_index(drop=True)
test_labels = test_labels.reset_index(drop=True)
stackPreds.columns = ['Ts_Valor','HR_Valor','QFE_Valor','h','d','m','y']
stackPreds = stackPreds[['Ts_Valor','HR_Valor','QFE_Valor']]

In [347]:
stackPreds.plot(subplots=True)
test_labels.plot(subplots=True)
plt.show()

In [348]:
x = ((stackPreds - test_df[['Ts_Valor','HR_Valor','QFE_Valor']][0:72]))
x.mean()

Ts_Valor    -1.019569
HR_Valor     8.477222
QFE_Valor    1.214352
dtype: float64

Como se puede observar el error promedio ha aumentado considerablemente en comparación al anteriormente expuesto, sin embargo, es un pronóstico indiscutiblemente más optimista que el presentado en la **iteración 2**, ocupando la **misma fórmula**. Este proceso de retroalimentación garantiza la autonomía del modelo durante largos periodos de tiempo y por lo tanto, está listo para ser implementado como servicio con capacidad predictiva básica.

Por último, se adjunta una gráfica donde se evidencia la diferencia de error promedio entre las predicciones autónomas y aquellas generadas mediante mapeo *'test_df(n) -> predicción(n+1)*

In [349]:
plt.plot(test_df['Ts_Valor'],label='testDF')
plt.plot(predRF[0:,0],label='predTestDF')
plt.plot(stackPreds['Ts_Valor'],label='predAutonoma')
plt.legend()
plt.grid()
plt.show()

# **Iteración seis: Modelo recurrente -LSTM-**


## Preparación de datos
**Selección de datos**: Ídem iteración anterior

**Integración**: Ídem iteración anterior

**Formateo**: Además de normalizar y descomponer la fecha de registro de forma tradicional, para cumplir con el objetivo de implementar un sistema de ventanas, se hace uso de un **generador de series de tiempo**

## Modelado
+ Fórmula: **Ts(n) ~ Ts(n-1) + Ts(n-2) + ... + Ts(n-24) + H + D + M** , en su versión multivariable (Ts+HR+QFE)(n)
+ Algoritmo seleccionado: LSTM con capas densas

In [350]:
df = pandas.read_csv("./dataPreprocessed.csv")
df = createTimeFeatures(df)
dates_df = df.pop('utc')
dates_df = df.pop('minute')
dates_df = df.pop('year')
#df

In [351]:
## Normalizado
train_mean = df.mean()
train_std = df.std()
df = (df - train_mean) / train_std

In [352]:
## Creación datasets de entrenamiento y testeo
lendf = len(df)
lendf = round(lendf*0.9)
train_df = df[0:lendf]
train_labels = df[['Ts_Valor','HR_Valor','QFE_Valor']][0:lendf]
#train_labels = df[0:lendf]
test_df = df[lendf:len(df)]
test_labels = df[['Ts_Valor','HR_Valor','QFE_Valor']][lendf:len(df)]
#test_labels = df[lendf:len(df)]

In [353]:
#test_df

In [354]:
train_df = train_df.to_numpy()
test_df = test_df.to_numpy()
train_labels = train_labels.to_numpy()
test_labels = test_labels.to_numpy()

In [355]:
winSize = 24 # passing events as 24hr windows
numFeatures = 6 # we are passing all available features, you can replace 'month' by 'cos'
train_gen = tf.keras.preprocessing.sequence.TimeseriesGenerator(train_df,train_labels,length=winSize)
test_gen = tf.keras.preprocessing.sequence.TimeseriesGenerator(test_df,test_labels,length=winSize)

In [356]:
np.set_printoptions(suppress=True)
X,y = train_gen[0]
print(f'Given the Array: \n{X[0]}')
print(f'Predict this y: \n {y[0]}')

Given the Array: 
[[ 0.87972626  0.16462007 -1.4166927  -1.66145972 -1.67374551 -1.51053581]
 [ 1.00995708 -0.38206422 -1.20653737 -1.51699757 -1.67374551 -1.51053581]
 [ 0.94484167 -0.38206422 -1.16450631 -1.37253542 -1.67374551 -1.51053581]
 [ 0.87972626 -0.38206422 -1.45872377 -1.22807327 -1.67374551 -1.51053581]
 [ 0.78205315 -0.24539315 -1.75294123 -1.08361112 -1.67374551 -1.51053581]
 [ 0.65182233 -0.10872208 -2.13122082 -0.93914898 -1.67374551 -1.51053581]
 [ 0.74949544 -0.38206422 -2.42543829 -0.79468683 -1.67374551 -1.51053581]
 [ 0.71693774 -0.38206422 -2.76168681 -0.65022468 -1.67374551 -1.51053581]
 [ 0.58670692 -0.24539315 -2.84574895 -0.50576253 -1.67374551 -1.51053581]
 [ 0.19601448  0.84797544 -2.55153148 -0.36130038 -1.67374551 -1.51053581]
 [ 0.22857218  0.71130437 -2.13122082 -0.21683823 -1.67374551 -1.51053581]
 [ 0.4239184   0.30129115 -1.83700336 -0.07237608 -1.67374551 -1.51053581]
 [ 0.87972626 -0.38206422 -1.6688791   0.07208606 -1.67374551 -1.51053581]
 [ 1.04

In [357]:
Given the Array: 
[[  21.6   68.  1005.6    0.     1.     1. ]
 [  22.    64.  1006.1    1.     1.     1. ]
 [  21.8   64.  1006.2    2.     1.     1. ]
 [  21.6   64.  1005.5    3.     1.     1. ]
 [  21.3   65.  1004.8    4.     1.     1. ]
 [  20.9   66.  1003.9    5.     1.     1. ]
 [  21.2   64.  1003.2    6.     1.     1. ]
 [  21.1   64.  1002.4    7.     1.     1. ]
 [  20.7   65.  1002.2    8.     1.     1. ]
 [  19.5   73.  1002.9    9.     1.     1. ]
 [  19.6   72.  1003.9   10.     1.     1. ]
 [  20.2   69.  1004.6   11.     1.     1. ]
 [  21.6   64.  1005.    12.     1.     1. ]
 [  22.1   63.  1004.9   13.     1.     1. ]
 [  22.5   65.  1004.6   14.     1.     1. ]
 [  22.9   63.  1004.3   15.     1.     1. ]
 [  23.3   62.  1003.6   16.     1.     1. ]
 [  23.2   63.  1003.3   17.     1.     1. ]
 [  24.    61.  1002.9   18.     1.     1. ]
 [  23.8   58.  1003.4   19.     1.     1. ]
 [  23.6   59.  1003.    20.     1.     1. ]
 [  23.1   63.  1002.8   21.     1.     1. ]
 [  22.5   67.  1002.9   22.     1.     1. ]
 [  22.5   67.  1003.8   23.     1.     1. ]]
Predict this y: 
 [  22.2   67.  1004.4    0.     2.     1. ]

SyntaxError: invalid syntax (<ipython-input-357-52ae9ed2bc1e>, line 1)

In [358]:
train_df[:26]

array([[ 0.87972626,  0.16462007, -1.4166927 , -1.66145972, -1.67374551,
        -1.51053581],
       [ 1.00995708, -0.38206422, -1.20653737, -1.51699757, -1.67374551,
        -1.51053581],
       [ 0.94484167, -0.38206422, -1.16450631, -1.37253542, -1.67374551,
        -1.51053581],
       [ 0.87972626, -0.38206422, -1.45872377, -1.22807327, -1.67374551,
        -1.51053581],
       [ 0.78205315, -0.24539315, -1.75294123, -1.08361112, -1.67374551,
        -1.51053581],
       [ 0.65182233, -0.10872208, -2.13122082, -0.93914898, -1.67374551,
        -1.51053581],
       [ 0.74949544, -0.38206422, -2.42543829, -0.79468683, -1.67374551,
        -1.51053581],
       [ 0.71693774, -0.38206422, -2.76168681, -0.65022468, -1.67374551,
        -1.51053581],
       [ 0.58670692, -0.24539315, -2.84574895, -0.50576253, -1.67374551,
        -1.51053581],
       [ 0.19601448,  0.84797544, -2.55153148, -0.36130038, -1.67374551,
        -1.51053581],
       [ 0.22857218,  0.71130437, -2.13122082, -0.

In [359]:
array([[  21.6,   68. , 1005.6,    0. ,    1. ,    1. ],
       [  22. ,   64. , 1006.1,    1. ,    1. ,    1. ],
       [  21.8,   64. , 1006.2,    2. ,    1. ,    1. ],
       [  21.6,   64. , 1005.5,    3. ,    1. ,    1. ],
       [  21.3,   65. , 1004.8,    4. ,    1. ,    1. ],
       [  20.9,   66. , 1003.9,    5. ,    1. ,    1. ],
       [  21.2,   64. , 1003.2,    6. ,    1. ,    1. ],
       [  21.1,   64. , 1002.4,    7. ,    1. ,    1. ],
       [  20.7,   65. , 1002.2,    8. ,    1. ,    1. ],
       [  19.5,   73. , 1002.9,    9. ,    1. ,    1. ],
       [  19.6,   72. , 1003.9,   10. ,    1. ,    1. ],
       [  20.2,   69. , 1004.6,   11. ,    1. ,    1. ],
       [  21.6,   64. , 1005. ,   12. ,    1. ,    1. ],
       [  22.1,   63. , 1004.9,   13. ,    1. ,    1. ],
       [  22.5,   65. , 1004.6,   14. ,    1. ,    1. ],
       [  22.9,   63. , 1004.3,   15. ,    1. ,    1. ],
       [  23.3,   62. , 1003.6,   16. ,    1. ,    1. ],
       [  23.2,   63. , 1003.3,   17. ,    1. ,    1. ],
       [  24. ,   61. , 1002.9,   18. ,    1. ,    1. ],
       [  23.8,   58. , 1003.4,   19. ,    1. ,    1. ],
       [  23.6,   59. , 1003. ,   20. ,    1. ,    1. ],
       [  23.1,   63. , 1002.8,   21. ,    1. ,    1. ],
       [  22.5,   67. , 1002.9,   22. ,    1. ,    1. ],
       [  22.5,   67. , 1003.8,   23. ,    1. ,    1. ],
       [  22.2,   67. , 1004.4,    0. ,    2. ,    1. ],
       [  22. ,   67. , 1005. ,    1. ,    2. ,    1. ]])

NameError: name 'array' is not defined

In [360]:
nnHM = tf.keras.Sequential()
#nnHM.add(tf.keras.layers.Dense(24,input_shape=(2,),activation='relu')) # this is the input shape if you pass 2 variables
# Adding more `lstm_units` just overfits more quickly.
nnHM.add(tf.keras.layers.Dense(168,activation='relu',input_shape=(winSize,numFeatures,)))
#nnHM.add(tf.keras.layers.LSTM(144,activation='relu',input_shape=(winSize,numFeatures,),return_sequences=True))
nnHM.add(tf.keras.layers.LSTM(24,return_sequences=True))
nnHM.add(tf.keras.layers.SimpleRNN(6,return_sequences=False))
nnHM.add(tf.keras.layers.Flatten())
nnHM.add(tf.keras.layers.Dense(3))

In [362]:
nnHM.compile(
    optimizer=tf.optimizers.Adam(),
    loss='mae', # 'mean_absolute_error',
    metrics=[tf.keras.metrics.MeanSquaredError()]
)
history = nnHM.fit_generator(
    train_gen,
    epochs=2,
    verbose=1,
    shuffle=False,
    validation_data=test_gen
)

# 221/221 [==============================] - 30s 136ms/step - loss: 0.1939 - mean_squared_error: 0.0804 - val_loss: 0.2382 - val_mean_squared_error: 0.1322

Epoch 1/2
221/221 [==============================] - 40s 151ms/step - loss: 0.4372 - mean_squared_error: 0.3353 - val_loss: 0.3554 - val_mean_squared_error: 0.2259
Epoch 2/2
221/221 [==============================] - 34s 155ms/step - loss: 0.2906 - mean_squared_error: 0.1564 - val_loss: 0.2935 - val_mean_squared_error: 0.1739


In [363]:
lastTrainBatch = train_df[-24:]
lastTrainBatch = np.array(lastTrainBatch)
# now reshape this lastTrainBatch to meet what input_shape the model expects
lastTrainBatch = lastTrainBatch.reshape((1,winSize,numFeatures))
lstmPred = nnHM.predict(lastTrainBatch,verbose=1)
lstmPred

1/1 [==============================] - 1s 720ms/step


array([[-0.17476374, -0.7801555 , -0.00289125]], dtype=float32)

In [364]:
x = nnHM.predict_generator(test_gen,verbose=1)
x.shape

/usr/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


25/25 [==============================] - 2s 51ms/step


(3114, 3)

In [365]:
# de-normalize
X = pandas.DataFrame(x)
#X.columns = ['Ts_Valor','HR_Valor','QFE_Valor','hour','day','month']
X.columns = ['Ts_Valor','HR_Valor','QFE_Valor']

In [366]:
varToPred = ['Ts_Valor','HR_Valor','QFE_Valor']
X = X*train_std[varToPred]+train_mean[varToPred]
X

,Ts_Valor,HR_Valor,QFE_Valor
0,18.290259,62.997636,1009.024455
1,18.185517,62.959661,1008.849968
2,17.615544,63.983202,1008.550531
3,18.415523,62.110045,1007.588852
4,18.250548,60.880337,1007.921384
...,...,...,...
3109,16.945654,70.603444,1010.435167
3110,16.475122,72.561553,1010.386923
3111,15.808300,75.956146,1010.611781
3112,15.915052,75.436123,1010.872343


In [367]:
test_df = df[lendf:len(df)]
test_labels = df[lendf:len(df)]
#test_df

In [368]:
# test_labels = test_labels*train_std[varToPred]+train_mean[varToPred]
test_labels = test_labels*train_std+train_mean
test_labels = test_labels.reset_index(drop=True)

In [369]:
### the offseted prediction works wonders -offset by 24 hours-
plt.plot(test_labels[['Ts_Valor']],label='testDF')
plt.plot(X[['Ts_Valor']],label='dense+lstm')
plt.grid()
plt.legend()
plt.show()

In [370]:
a = (X - test_labels).mean()
a

HR_Valor    -0.543562
QFE_Valor    0.050571
Ts_Valor    -0.278398
day               NaN
hour              NaN
month             NaN
dtype: float64

## Evaluación

Ídem a iteraciones anteriores, estos valores de error promedio no son suficientes para asegurar la calidad del modelo, se debe evaluar su capacidad de retroalimentación en tiempo real, por lo tanto se genera un nuevo proceso de retroalimentación

In [371]:
now = datetime.datetime.now()
now = pandas.to_datetime('2021-07-29 09:00:00')
now = pandas.to_datetime('2021-03-23 06:00:00')
df3 = train_df[-24:]
stackPreds = pandas.DataFrame()
lastTrainBatch = train_df[-24:]
# now reshape this lastTrainBatch to meet what input_shape the model expects
lastTrainBatch = lastTrainBatch.reshape((1,winSize,numFeatures))
x = nnHM.predict(lastTrainBatch)

In [372]:
def norm(value,index):
    #value = (value - modelMean[index]) / modelStd[index]
    value = (value - train_mean[index]) / train_std[index]
    return value

In [373]:
for i in range(0,72):
    delta = now + datetime.timedelta(0,i*3600)
    #temp = np.array([x,y,z,delta.hour,delta.day,delta.month],dtype="float32")
    # temp = np.array([x,y,z,norm(delta.hour,3),norm(delta.day,4),norm(delta.month,5)],dtype="float32")
    temp = np.array([x[0,0],x[0,1],x[0,2],norm(delta.hour,3),norm(delta.day,4),norm(delta.month,5)],dtype="float32")
    stackPreds = stackPreds.append(pandas.DataFrame(temp).transpose())
    # df3 = np.vstack((df3[0],temp))
    # df4 = np.vstack((df4,temp))
    cde = np.reshape(df3,(24,6))
    df3 = np.vstack((cde,temp))
    df3 = np.delete(df3, (0), axis=0)
    df3 = np.reshape(df3,(-1,24,6))
    x = nnHM.predict(df3)

In [374]:
stackPreds = stackPreds.reset_index(drop=True)
stackPreds.columns = ['Ts_Valor','HR_Valor','QFE_Valor','hour','day','month']
stackPreds = stackPreds*train_std+train_mean
stackPreds

,Ts_Valor,HR_Valor,QFE_Valor,hour,day,month
0,18.361166,61.087231,1008.963706,6.0,23.0,3.0
1,18.087312,60.659646,1009.144957,7.0,23.0,3.0
2,17.859438,60.006028,1009.346071,8.0,23.0,3.0
3,17.760234,59.101864,1009.324442,9.0,23.0,3.0
4,17.745547,58.306564,1009.413465,10.0,23.0,3.0
...,...,...,...,...,...,...
67,15.645422,71.734354,1010.906494,1.0,26.0,3.0
68,15.421414,72.860856,1011.048804,2.0,26.0,3.0
69,15.152991,74.258594,1010.824200,3.0,26.0,3.0
70,14.902286,75.537915,1010.458525,4.0,26.0,3.0


In [375]:
predTest = nnHM.predict(test_gen,verbose=1)
predTest = predTest*train_std[varToPred].to_numpy()+train_mean[varToPred].to_numpy()

25/25 [==============================] - 2s 62ms/step


In [376]:
df = pandas.read_csv("./dataPreprocessed.csv")
df = createTimeFeatures(df)
dates_df = df.pop('utc')
dates_df = df.pop('minute')
dates_df = df.pop('year')
test_df = df[lendf:len(df)]
test_labels = df[['Ts_Valor','HR_Valor','QFE_Valor']][lendf:len(df)]
test_labels = test_labels.reset_index(drop=True)

In [377]:
test_df

,Ts_Valor,HR_Valor,QFE_Valor,hour,day,month
28243,19.4,63.0,1008.9,6,23,3
28244,19.4,62.0,1008.6,7,23,3
28245,19.9,62.0,1008.0,8,23,3
28246,19.9,59.0,1007.9,9,23,3
28247,18.5,65.0,1008.0,10,23,3
...,...,...,...,...,...,...
31376,16.9,72.0,1009.9,19,31,7
31377,16.1,76.0,1010.1,20,31,7
31378,15.9,76.0,1010.5,21,31,7
31379,15.6,77.0,1011.2,22,31,7


In [378]:
plt.plot(test_labels['Ts_Valor'], label='testLabels')
plt.plot(predTest[0:,0],label='predTestDF')
plt.plot(stackPreds['Ts_Valor'],label='predAutonoma')
plt.legend()
plt.grid()
plt.show()

In [379]:
plt.plot(test_labels['HR_Valor'], label='testLabels')
plt.plot(predTest[0:,1],label='predTestDF')
plt.plot(stackPreds['HR_Valor'],label='predAutonoma')
plt.legend()
plt.grid()
plt.show()

In [380]:
a = stackPreds - test_labels[:72]
a.mean()

HR_Valor     7.588925
QFE_Valor    1.922293
Ts_Valor    -4.633723
day               NaN
hour              NaN
month             NaN
dtype: float64

In [ ]:
# nnHM.save("../../backup/lstmMIMO")

In [ ]:
nnHM = tf.keras.models.load_model("../../backup/lstmMIMO")